In [ ]:
import os
import numpy as np
import pandas as pd
import string
import time
import urllib.request
import zipfile
import torch

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 54.2 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 54.3 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=92a7913248b46559efaf98c8d2373962aeb3a6652e4b9c9ad1d204fb92a78683
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
data1 = pd.read_csv(r'/content/Nigeria2019_ACLED_Extract.csv')
data2 = pd.read_csv(r'/content/2020-01-01-2022-04-01-Western_Africa-Nigeria.csv')
df = pd.concat([data1, data2], axis=0)

In [ ]:
df = df[['notes', 'source']]
label_encoder = preprocessing.LabelEncoder()
df['source']= label_encoder.fit_transform(df['source'])
from sklearn.model_selection import train_test_split
X = df['notes']
y = df['source']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = X_train.tolist()
X_test = X_test.tolist()

In [ ]:
# Load pre-trained model
senttrans_model = SentenceTransformer('all-MiniLM-L6-v2',device=device)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# Create embeddings for training set text
X_train = X_train
X_train = [senttrans_model.encode(doc) for doc in X_train]

# Create embeddings for test set text
X_test = X_test
X_test = [senttrans_model.encode(doc) for doc in X_test]

In [ ]:
# Train a classification model using logistic regression classifier
y_train = pd.Series(y_train)
logreg_model = LogisticRegression(solver='saga')
logreg_model.fit(X_train,y_train)
preds = logreg_model.predict(X_train)
acc = sum(preds==y_train)/len(y_train)
print('Accuracy on the training set is {:.3f}'.format(acc))

Accuracy on the training set is 0.208


In [ ]:
# Evaluate performance on the test set
y_test = pd.Series(y_test)
preds = logreg_model.predict(X_test)
acc = sum(preds==y_test)/len(y_test)
print('Accuracy on the test set is {:.3f}'.format(acc))

Accuracy on the test set is 0.157


In [ ]:
import joblib
file1 = 'logreg_model.joblib'
joblib.dump(logreg_model, file1)

['logreg_model.joblib']

In [ ]:
file2 = 'label_encoder.joblib'
joblib.dump(label_encoder, file2)

['label_encoder.joblib']

In [ ]:
file3 = 'senttrans_model.joblib'
joblib.dump(senttrans_model, file3)

['senttrans_model.joblib']